# 서론 

* 파이썬은 느리다... 

* 파이썬이 느린 이유가 몇가지 있겠지만, 하나하나씩 읽어 내려가기 위해 쓰레드를 하나만 사용하도록 잠금을 걸어 놓아서 병렬처리가 안된다는 단점이 하나가 있고,

* 하나는 변수를 동적으로 선언하기 때문에 참조하는데에 많은 시간이 걸린다는 것이 있다. 

* 이때문에 파이썬에서 한번씩 사용해본 넘파이나 텐서플로우 같은 라이브러리들은 모두 C 언어를 직접적으로 가져와서 사용하거나 래핑을 해서 사용한다. 

* C 언어로 작성된 코드를 파이썬으로 가져오기 위해서 여러 방법들이 존재하는데, 이 글에서는 그중에서 SWIG라는 프로그램이 존재한다. 

* 최신 업데이트가 2020년 인것으로 보아 활발한 커뮤니티를 가지고 있지는 않지만, 워낙 전통적으로 많이 사용되어져 왔던 프로그램이기 때문에 파이썬을 사용하는 유저로써 알아볼 가치가 있다고 생각한다. 

# SWIG 설치

* SWIG를 설치해주도록 하자.

```bash
$ sudo apt-get install swig
```

## 바꿀 C 파일.


**example.c**

이 c 파일을 파이썬에서 참조할수 있도록 할 것이다. 

```c
 /* File : example.c */
 
 #include <time.h>
 double My_variable = 3.0;

// 팩토리얼 연산
 int fact(int n) {
     if (n <= 1) return 1;
     else return n*fact(n-1);
 }
 
 // 단순한 나누기 연산
 int my_mod(int x, int y) {
     return (x%y);
 }
 	

// 시간 가져오는 함수
 char *get_time()
 {
     time_t ltime;
     time(&ltime);
     return ctime(&ltime);
 }
 
 
```
* 모르겠으면 아 그냥 그렇구나 하고 넘기도록 하자. 사실은 나도 잘 모른다.

* 다른 점은 파이썬은 타입을 선언해줄 필요가 없다는 점이고, C에서는 변수를 선언할때 항상 변수의 타입에 대해서 선언해주어야 한다는 점이다. 

* 파이썬은 동적인 타입선언이고, C/C++은 정적인 타입선언이다.




## example.i

SWIG가 참조해야할 인터페이스 파일을 하나 생성해야 한다. 



```
/* example.i */
 %module example
 %{
 /* 헤더파일과 함수들을 이곳에 넣어주도록 하자. */
 extern double My_variable;
 extern int fact(int n);
 extern int my_mod(int x, int y);
 extern char *get_time();
 %}
 
 extern double My_variable;
 extern int fact(int n);
 extern int my_mod(int x, int y);
 extern char *get_time();
```

## Python 모듈 생성

### 방법 1. gcc로 컴파일 후, so 파일 빌드.



#### Step 1.
```
% swig -python example.i
% gcc -c example.c example_wrap.c \
        -I/usr/local/include/python2.7
% ld -shared example.o example_wrap.o -o _example.so 
```

* 하나하나씩 실행하도록 하자. 
<br>
</br>
* 컴파일을 해주는 gcc의 옵션인 -c에서 swig로 만든 파일과 우리의 계산 코드를 컴파일 해보면.

```
% gcc -c example.c example_wrap.c 
```
Python.h가 없다고 오류를 gcc에서 낸다.


그렇다면... Python.h가 있는 곳을 찾아서 헤메야 할것 같다는 생각이 자연스레든다...


#### Step 2.

* SWIG의 튜토리얼 문서에는 `gcc -c example.c example_wrap.c \-I/usr/local/include/python2.7` 이 명령어를 실행시키라고 설명해주고 있다. 

* `gcc -c () ()`는 두 파일을 컴파일하는 명령어이고, 뒤의 경로는 무언가를 참조하고 있는 경로로 보여진다. 

* 우리들의 오류였던 
```
example_wrap.c:149:11: fatal error: Python.h: No such file or directory
  # include <Python.h>
            ^~~~~~~~~~
```
은 아마도 참조해야할 Python.h가 있는 경로를 제대로 설정하지 않아서 인것 처럼 보인다. 

* 그렇다면 Python.h 파일은 뭐 하는 파일일까...
* <https://github.com/python/cpython/tree/main/Include>

* 위 링크가 깃허브 Python.h 파일의 주소이다. 

* 파이썬은 Include라는 폴더에 설치가 되나보다. 

* 일반 리눅스에서는 /usr/local/include/python2.7/Python.h 으로 경로 설정이 되지만, 어째서인지 코랩에서는 /usr/include/python3.7/Python.h 으로 다운로드가 되어있다. (버전차이는 여러버전이 있을수 있으니... 그렇다치고...)

* 문제는 다 해결 되었다. (인줄 알았다.....)
```
$ gcc -c example.c example_wrap.c \
        -I/usr/include/python3.7
```
를 실행시켜주면 래핑된 컴파일 파일이 생성이 된다.

```bash 
$ ls 

example.c example.i example.py example.o example_wrap.c example_wrap.o
```

#### Step 3.

* 이 다음이 문제인데... 

```
$ ld -shared example.o example_wrap.o -o _example.so
```

* -fPIC을 컴파일 시에 명령어에 입력하라는 오류와 함께 멈추어 버렸다. 정말 여러가지를 시도해봤지만, 조금더 쉬운 방법으로 성공했으니 이 방법은 방치 하는 것으로.... 결정했다....

### 방법 2.  setup.py 사용하기.

#### Step 1.

* 우선 방법 1과 똑같이 `% swig -python example.i`를 실행시켜서 example.py와 example_wrap.c 파일을 생성하도록 하자.

```       

#### Step 2.

* setup.py를 다음과 같이 만들고 그 파일을 실행시키도록 하자.


```python
#!/usr/bin/env python

"""
setup.py file for SWIG example
"""

from distutils.core import setup, Extension

# Extension 안에 argument로 output파일 이름과, 우리들의 c 파일을 넣어주면 된다. 
example_module = Extension('_example',
                           sources=['example_wrap.c', 'example.c'],
                           )

                           
# setup 안에는 정보와 모듈의 이름이 들어간다. 맞춰주도록 하자. 
setup (name = 'example',
       version = '0.1',
       author      = "SWIG Docs",
       description = """Simple swig example from docs""",
       ext_modules = [example_module],
       py_modules = ["example"],
       )
```


```
$ python setup.py build_ext --inplace
```

이렇게 실행시키면 example 이라는 모듈을 사용할수 있게 된다.

# 결과 비교

* example_py.py 파일을 하나 준비해서 같은 팩토리얼 함수를 작성하자.

```python
def fact(x):
  if x == 1:
    return x
  else:
    return x * fact(x-1) 
```



In [ ]:
import example
import example_py
from time import time

* 우리가 SWIG로 만든 모듈과 파이썬 모듈로 비교해보도록 하자.

In [ ]:
iter_num = 100000

start = time()
for i in range(0,iter_num):
    example.fact(5)
end = time()
print(f'SWIG time {round(end -start, 2)} sec')

swig_time = end - start

python_start = time()
for i in range(0,iter_num):
    example_py.fact(5)
python_end = time()
print(f'Python time {round(python_end - python_start, 2)} sec')

python_time = python_end - python_start

print(f'{round(python_time/swig_time , 2)} times faster')

SWIG time 0.02 sec
Python time 0.07 sec
3.38 times faster


* 거의 3배나 차이가 나는 것을 볼수 있다. 

* 물론 어떠한 알고리즘이며 어떠한 로직인지 봐야 하겠지만, 이 정도면 상당히 차이가 많이나는 것을 볼수 있다. 